First we will use pyscf to create PySCF objects

In [1]:
from pyscf import gto, scf

# Build mols

h2o_mol = gto.M(atom='''
  O     0.00000000    -0.00001441    -0.34824012
  H    -0.00000000     0.76001092    -0.93285191
  H     0.00000000    -0.75999650    -0.93290797
        ''', basis='6-31G')

cl2_mol = gto.M(
    verbose = 0,
    atom='''
Cl           0.00000000 0.00000000 0.00000000
Cl           0.00000000 0.00000000 2.00000000
''',
    basis='6-31G', spin=0)

# Build and run RHF for water, UHF for cl2
h2o = scf.RHF(h2o_mol)
h2o.kernel()

cl2 = scf.UHF(cl2_mol)
cl2.kernel()

converged SCF energy = -75.9840968770105


-918.8634251135154

Initializing a RT_SCF object in TiDES

1. Create instance of RT_SCF using an SCF object, time step, and simulation time as required arguments.

Optional RT_SCF parameters

1.  filename
    - prints output file to this file (if filename is left as default value of none, output will go to stdout)
3.  prop
    - Runge-Kutta 4 - 'rk4'
    - Magnus Step (MMUT) - 'magnus_step'
    - Interpolated Magnus - 'magnus_interpol'
    - Default is magnus_interpol
5.  frequency
    - observables are printed every "frequency" number of electronic steps. Default is 1
7.  orth
    - orthogonalization matrix. Default is none, which will use canonical orthogonalization
9.  chkfile
    - specifies chkfile for restarting a calculation. See Chkfile folder in examples for how to use
11.  verbose
    - specifies verbosity of output file. Default is 3

In [2]:
from tides import rt_scf
rt_h2o = rt_scf.RT_SCF(h2o, 0.2, 10, filename=None, prop='magnus_interpol', frequency=1, orth=None, chkfile=None, verbose=3)

The RT_Ehrenfest class is a child class of RT_SCF, and has additional optional keyword arguments and functionality.

1. Ne_step
    - This determines the frequency in which nuclei positions/velocities are updated versus electronic steps. Default is 10
2. N_step
    - This determines the frequency in which nuclear forces are updated versus position/velocity updates. Default is 10
4. get_mo_coeff_print
    - Only for printing mo_occ observable with moving nuclei - function which takes rt_ehrenfest as argument, and calculates orbitals to calculate populations in. By defauly, mo_occ will be printed in the ground state SCF orbitals of the passed in SCF object, as the nuclear coordinates evolve in time (this will keep the original spin multiplicity).

In [3]:
from tides import rt_ehrenfest
rt_cl2 = rt_ehrenfest.RT_Ehrenfest(cl2, 0.5, 5, verbose=5, Ne_step=1, N_step=1, get_mo_coeff_print=None)

Declare observables by updating the dictionary attribute of the RT_SCF or RT_Ehrenfest object to be True

1. Energy - 'energy'
     - For Ehrenfest: atomic kinetic energies - verbose > 3
3. Dipole - 'dipole'
4. Quadrupole - 'quadrupole'
5. Electronic Charge - 'charge'
6. Mulliken Charge - 'atom_charge' or 'mulliken_charge' or 'mulliken_atom_charge'
7. Hirshfeld Charge - 'hirsh_charge' or 'hirsh_atom_charge'
8. Magnetization - 'mag'
9. Hirshfeld Magnetization - 'hirsh_mag' or 'hirsh_atom_mag'
10. MO occupations - 'mo_occ'
11. Nuclei (For Ehrenfest) - 'nuclei'
    - Positions
    - Velocities - verbose > 3
    - Forces - verbose > 4

In [4]:
rt_h2o.observables['dipole'] = True
rt_cl2.observables['nuclei'] = True

Adding external fields

1. Create instance of a field class
2. Add field to TiDES calculation using the add_potential() method.

In [5]:
from tides.rt_vapp import ElectricField

delta_field = ElectricField('delta', [0.0001, 0.0001, 0.0001])

rt_h2o.add_potential(delta_field)

Start propagation by calling the kernel() method of the RT_SCF class.

In [6]:
rt_h2o.kernel()

Beginning Propagation For: 

	 Object Type: RHF
	 Basis Set: 6-31G
	 Mol Length: 3


Propagation Settings: 

	 Real-Time SCF
	 Integrator: magnus_interpol
	 Max time (AU): 10
	 Time step (AU): 0.2
	 Observables: 

 	 	 dipole

Applied Potentials: 

	 	 ElectricField



Current Time (AU): 0.00000000 

Total Dipole Moment [X, Y, Z] (AU): 1.8579945514146294e-17 3.81423866103786e-05 -1.032718607751594 



Current Time (AU): 0.20000000 

Total Dipole Moment [X, Y, Z] (AU): 1.928542699639917e-06 4.061775693750547e-05 -1.0327168074099093 



Current Time (AU): 0.40000000 

Total Dipole Moment [X, Y, Z] (AU): 7.922881538295069e-06 5.2247320304801836e-05 -1.0327088260121835 



Current Time (AU): 0.60000000 

Total Dipole Moment [X, Y, Z] (AU): 1.2067800488047167e-05 6.03981830334388e-05 -1.0327032302999655 



Current Time (AU): 0.80000000 

Total Dipole Moment [X, Y, Z] (AU): 1.5088074933343105e-05 6.734435411290263e-05 -1.0326984443027145 



Current Time (AU): 1.00000000 

Total Dipole Mome

Non-equilibrium conditions can be created by modifying attributes of the SCF object or the RT_SCF/RT_Ehrenfest object before calling the kernel() function.

For the case of starting an Ehrenfest simulation with initial velocities, modify the nuc attribute of the ehrenfest class

In [7]:
import numpy as np
from pyscf import gto, scf
from tides import rt_ehrenfest

# Reset the cl2_mol and SCF object (for when you want to re-run this box)
cl2_mol = gto.M(
    verbose = 0,
    atom='''
Cl           0.00000000 0.00000000 0.00000000
Cl           0.00000000 0.00000000 2.00000000
''',
    basis='6-31G', spin=0)

cl2 = scf.UHF(cl2_mol)
cl2.kernel()

rt_cl2 = rt_ehrenfest.RT_Ehrenfest(cl2, 0.5, 5, verbose=5, Ne_step=1, N_step=1, get_mo_coeff_print=None)
rt_cl2.observables['nuclei'] = True

# Set initial kinetic energy of 1 eV for rt_cl2
init_eV = 1.0

# Convert to velocity (in au, not Angstroms) assuming Cl atoms are moving at same speed apart from each other (in the z-direction)
# Cl mass ~= 35 * 1836 m_e = 64260 m_e
# T = \frac{mv^2}{2}

init_velo = np.sqrt((init_eV/(2*27.2114))*2/64260)

# First index of rt_cl2.nuc.vel is the atom, second index is the direction (x, y, z)
rt_cl2.nuc.vel[0,2] = -1 * init_velo
rt_cl2.nuc.vel[1,2] = init_velo

# Now run
rt_cl2.kernel()

Beginning Propagation For: 

	 Object Type: UHF
	 Basis Set: 6-31G
	 Mol Length: 2


Propagation Settings: 

	 Real-Time SCF w/ Ehrenfest Dynamics
	 Integrator: magnus_interpol
	 Max time (AU): 5
	 Time step (AU): 0.5
	 Nuclear Position Update Frequency: 1
	 Nuclear Force Update Frequency: 1
	 Observables: 

 	 	 nuclei



Current Time (AU): 0.00000000 

Nuclear Coordinates (Angstrom):
 Cl 	 0.00000000000	0.00000000000	0.00000000000
 Cl 	 0.00000000000	0.00000000000	2.00000000000
 
Nuclear Velocities (Angstrom / AU):
 Cl 	 0.00000000000	0.00000000000	-0.00040017995
 Cl 	 0.00000000000	0.00000000000	0.00040017995
 
Nuclear Forces (AU):
 Cl 	 -0.00000000000	0.00000000000	-0.07151542180
 Cl 	 0.00000000000	-0.00000000000	0.07151542180
 


Current Time (AU): 0.50000000 

Nuclear Coordinates (Angstrom):
 Cl 	 -0.00000000000	0.00000000000	-0.00020016359
 Cl 	 0.00000000000	-0.00000000000	2.00020016359
 
Nuclear Velocities (Angstrom / AU):
 Cl 	 0.00000000000	-0.00000000000	-0.00040047331
 Cl

Custom observables can be added by modifying the private _observables_functions attribute of the RT_SCF object

1. Write a function that takes in rt_scf and den_ao (time-dependent 1RDM) as arguments and calculates an observable. Save that observable as a private attribute of the rt_scf class
2. Write another function that takes rt_scf as an argument and prints the custom observable to the output file, formatted however you would like

In [8]:
# First reset the rt_h2o object (don't need to reset mol since nuclei are fixed in RT_SCF)
from pyscf import scf
from tides import rt_scf 

h2o = scf.RHF(h2o_mol)
h2o.kernel()

rt_h2o = rt_scf.RT_SCF(h2o, 0.2, 10, filename=None, prop='magnus_interpol', frequency=1, orth=None, chkfile=None, verbose=3)

# Define blank custom observable
def get_custom_observable(rt_scf, den_ao):
    rt_scf._custom_observable = None

def print_custom_observable(rt_scf):
    rt_scf._log.note(f'HERE IS THE CUSTOM OBSERVABLE: {rt_scf._custom_observable}')


# Add these functions to rt_h2o._observables_functions dictionary and declare the custom observable in rt_h2o.observables dictionary.
# Make sure to keep the key name consistent

rt_h2o._observables_functions['custom'] = [get_custom_observable, print_custom_observable]

rt_h2o.observables['custom'] = True

rt_h2o.kernel()


converged SCF energy = -75.9840968770105
Beginning Propagation For: 

	 Object Type: RHF
	 Basis Set: 6-31G
	 Mol Length: 3


Propagation Settings: 

	 Real-Time SCF
	 Integrator: magnus_interpol
	 Max time (AU): 10
	 Time step (AU): 0.2
	 Observables: 

 	 	 custom



Current Time (AU): 0.00000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 0.20000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 0.40000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 0.60000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 0.80000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 1.00000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 1.20000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 1.40000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 1.60000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 1.80000000 

HERE IS THE CUSTOM OBSERVABLE: None


Current Time (AU): 2.00000000 

Custom external fields can be defined as classes with the method "calculate_potential(self, rt_scf)" that returns a matrix of same dimensionality as the Fock matrix with the external potential terms in the non-orthogonal atomic orbital basis.

In [9]:
# First reset the rt_h2o object (don't need to reset mol since nuclei are fixed in RT_SCF)
from pyscf import scf
from tides import rt_scf 

h2o = scf.RHF(h2o_mol)
h2o.kernel()

rt_h2o = rt_scf.RT_SCF(h2o, 0.2, 10, filename=None, prop='magnus_interpol', frequency=1, orth=None, chkfile=None, verbose=3)

# Define custom field class. Make sure the calculate_potential method is written correctly and return matrix of correct shape (IN THE NON-ORTHOGONAL AO BASIS)
class CUSTOM:
    def __init__(self):
        pass
        
    def calculate_potential(self, rt_scf):
        return np.zeros(rt_scf.fock_ao.shape)


# Create instance of custom field
custom_instance = CUSTOM()

# Add field to rt_h2o
rt_h2o.add_potential(custom_instance)

rt_h2o.kernel()

converged SCF energy = -75.9840968770105
Beginning Propagation For: 

	 Object Type: RHF
	 Basis Set: 6-31G
	 Mol Length: 3


Propagation Settings: 

	 Real-Time SCF
	 Integrator: magnus_interpol
	 Max time (AU): 10
	 Time step (AU): 0.2
	 Observables: 


Applied Potentials: 

	 	 CUSTOM



Current Time (AU): 0.00000000 



Current Time (AU): 0.20000000 



Current Time (AU): 0.40000000 



Current Time (AU): 0.60000000 



Current Time (AU): 0.80000000 



Current Time (AU): 1.00000000 



Current Time (AU): 1.20000000 



Current Time (AU): 1.40000000 



Current Time (AU): 1.60000000 



Current Time (AU): 1.80000000 



Current Time (AU): 2.00000000 



Current Time (AU): 2.20000000 



Current Time (AU): 2.40000000 



Current Time (AU): 2.60000000 



Current Time (AU): 2.80000000 



Current Time (AU): 3.00000000 



Current Time (AU): 3.20000000 



Current Time (AU): 3.40000000 



Current Time (AU): 3.60000000 



Current Time (AU): 3.80000000 



Current Time (AU): 4.0000000

To add static potentials for RT_SCF objects, you can override the get_hcore() method of the SCF object, adding the static potential matrix.

#### NOTE: THIS PROCEDURE WILL NOT WORK FOR RT_EHRENFEST CALCULATIONS, SINCE 1-ELECTRON TERMS WILL NEED TO BE RE-CALCULATED WITH NUCLEAR GEOMETRY UPDATES

In [10]:
import numpy as np

# Create new RHF object for h2o to be modified 
h2o_modified = scf.RHF(h2o_mol)
h2o_modified.kernel()

hcore = h2o_modified.get_hcore()

# Define external static field, add to hcore
static_field = np.zeros(hcore.shape)

hcore_modified = hcore + static_field

# Override get_hcore() with modified hcore
h2o_modified.get_hcore = lambda *args: hcore_modified

from tides import rt_scf
rt_h2o_modified = rt_scf.RT_SCF(h2o_modified, 0.2, 10, filename=None, prop='magnus_interpol', frequency=1, orth=None, chkfile=None, verbose=3)

rt_h2o_modified.kernel()

converged SCF energy = -75.9840968770105
Beginning Propagation For: 

	 Object Type: RHF
	 Basis Set: 6-31G
	 Mol Length: 3


Propagation Settings: 

	 Real-Time SCF
	 Integrator: magnus_interpol
	 Max time (AU): 10
	 Time step (AU): 0.2
	 Observables: 




Current Time (AU): 0.00000000 



Current Time (AU): 0.20000000 



Current Time (AU): 0.40000000 



Current Time (AU): 0.60000000 



Current Time (AU): 0.80000000 



Current Time (AU): 1.00000000 



Current Time (AU): 1.20000000 



Current Time (AU): 1.40000000 



Current Time (AU): 1.60000000 



Current Time (AU): 1.80000000 



Current Time (AU): 2.00000000 



Current Time (AU): 2.20000000 



Current Time (AU): 2.40000000 



Current Time (AU): 2.60000000 



Current Time (AU): 2.80000000 



Current Time (AU): 3.00000000 



Current Time (AU): 3.20000000 



Current Time (AU): 3.40000000 



Current Time (AU): 3.60000000 



Current Time (AU): 3.80000000 



Current Time (AU): 4.00000000 



Current Time (AU): 4.2000000